In [172]:
import networkx as nx
import matplotlib.pyplot as plt
import scipy as sci
import numpy as np
import numpy.linalg as linalg
import numpy.random as rand
import pandas as pd
from platform import node
from cdlib.classes.node_clustering import NodeClustering
import community.community_louvain as community_louvain
import cdlib as cd
from collections import defaultdict
from collections import Counter
import networkx as nx
import cdlib.algorithms as algo
import cdlib.viz as viz
import matplotlib.pyplot as plt
import pyvis
from pyvis.network import Network
import random

from functions import louvain

In [25]:
def get_degrees(edges):
    counter = Counter(edges)
    return counter.most_common()

In [195]:
# reading in the data, creating the graph

hero_edges = pd.read_csv('hero-network.csv')
nodes = pd.read_csv('nodes.csv')
edges = pd.read_csv('edges.csv')

hero_edges['hero1'].str.strip()
hero_edges['hero2'].str.strip()

hero_graph = nx.from_pandas_edgelist(hero_edges, 'hero1', 'hero2', create_using=nx.MultiGraph())

In [196]:
# get degrees of each node

hero_nodes = []
for x in range(len(nodes['node'])):
    if nodes['type'][x] != 'comic':
        hero_nodes.append(nodes['node'][x])

stubs1 = list(hero_edges['hero1'])
stubs2 = list(hero_edges['hero2'])

stubs = stubs1 + stubs2

hero_degrees = get_degrees(stubs)
degrees = [x[1] for x in hero_degrees]
heroes_sorted = [x[0] for x in hero_degrees]

In [197]:
MarvelCluster = louvain(hero_graph)

In [198]:
print(len(MarvelCluster.communities))
for i in range(len(MarvelCluster.communities)):
    MarvelCluster.communities[i] = sorted(MarvelCluster.communities[i], key = heroes_sorted.index)

18


In [199]:
coms = pd.DataFrame(MarvelCluster.communities)
coms = coms.transpose()



In [155]:
coms[2]

0       SPIDER-MAN/PETER PAR
1          JAMESON, J. JONAH
2       WATSON-PARKER, MARY 
3       DAREDEVIL/MATT MURDO
4       BLACK PANTHER/T'CHAL
                ...         
1315                    None
1316                    None
1317                    None
1318                    None
1319                    None
Name: 2, Length: 1320, dtype: object

In [200]:
r = lambda: random.randint(0,255)

com_graph = nx.MultiGraph()
pos = nx.spring_layout(com_graph, scale = 500)

for community in coms.transpose().to_numpy():
    communityColorString = '#%02X%02X%02X' % (r(),r(),r())
    
    # Loop over each node 
    for node in list(community):
        if node == None: 
            continue
        else:
            com_graph.add_node(node, label = node, color=communityColorString)


com_graph.add_edges_from(random.sample(list(hero_edges.itertuples()), 5000))

['BETA RAY BILL',
 'JUSTICE II/VANCE AST',
 'VISALIA',
 'MENTOR/ALARS [ETERNA',
 'CHI-CHEE',
 'STARSHINE II/BRANDY ',
 'PSYLOCKE/ELISABETH B',
 'ELSIE DEE',
 'RAMIREZ, JAVIER',
 'GAMBIT/REMY LEBEAU ',
 'AIREO/AEOLUS [INHUMA',
 'SIRYN/THERESA ROURKE',
 'WU, LEIKO',
 'IRONCLAD',
 'SHADOWCAT/KATHERINE ',
 'DRAKE, FRANKLIN',
 'MAGIK/ILLYANA RASPUT',
 'JUBILEE/JUBILATION L',
 'DELILAH',
 'MARVEL GIRL/JEAN GRE',
 'MOCKINGBIRD/DR. BARB',
 'CLOUD',
 'DARKSTAR/LAYNIA SERG',
 'HARNESS/ERIKA BENSON',
 'DR. STRANGE/STEPHEN ',
 'VULTURE II/BLACKIE D',
 'CUT',
 'QUASAR III/WENDELL V',
 'PENANCE/MONET ST. CR',
 'MEPHISTO',
 'WOLFSBANE/RAHNE SINC',
 "KARMA/XI'AN COY MANH",
 'DR. STRANGE/STEPHEN ',
 'CYCLOPS/SCOTT SUMMER',
 'DELILAH',
 'WOZ/DEWOZ',
 'CARPENTER, RACHEL',
 'ARAGORN',
 'STACY, CAPT. GEORGE',
 'JETSTREAM/HAROUD IBN',
 'CAPTAIN AMERICA',
 'M',
 'DEADPOOL/JACK/WADE W',
 'SPIDER-MAN/PETER PAR',
 'CYCLOPS/SCOTT SUMMER',
 'WONDER MAN/SIMON WIL',
 'HYPERION II',
 'SPIDER-MAN/PETER PAR',
 'CADUCE

In [ ]:

net = Network(height="1000px", width="100%", bgcolor="#222222", font_color="white", notebook=True, directed = False, cdn_resources='in_line')

net.barnes_hut()
net.from_nx(com_graph)

net.show("Marvel2.html")

In [202]:
coms_map = dict()
for sub in MarvelCluster.communities:
    for node in sub:
        coms_map[node] = sub[0]

print(coms_map)


graph_contracted = community_louvain.induced_graph(coms_map, hero_graph, weight='weight')
graph_contracted.remove_edges_from(nx.selfloop_edges(graph_contracted))

{'WOLVERINE/LOGAN ': 'WOLVERINE/LOGAN ', 'BEAST/HENRY &HANK& P': 'WOLVERINE/LOGAN ', 'CYCLOPS/SCOTT SUMMER': 'WOLVERINE/LOGAN ', 'STORM/ORORO MUNROE S': 'WOLVERINE/LOGAN ', 'COLOSSUS II/PETER RA': 'WOLVERINE/LOGAN ', 'PROFESSOR X/CHARLES ': 'WOLVERINE/LOGAN ', 'MARVEL GIRL/JEAN GRE': 'WOLVERINE/LOGAN ', 'ANGEL/WARREN KENNETH': 'WOLVERINE/LOGAN ', 'ICEMAN/ROBERT BOBBY ': 'WOLVERINE/LOGAN ', 'NIGHTCRAWLER/KURT WA': 'WOLVERINE/LOGAN ', 'ROGUE /': 'WOLVERINE/LOGAN ', 'SHADOWCAT/KATHERINE ': 'WOLVERINE/LOGAN ', 'CANNONBALL II/SAM GU': 'WOLVERINE/LOGAN ', 'PSYLOCKE/ELISABETH B': 'WOLVERINE/LOGAN ', 'WOLFSBANE/RAHNE SINC': 'WOLVERINE/LOGAN ', 'GAMBIT/REMY LEBEAU ': 'WOLVERINE/LOGAN ', 'MAGNETO/MAGNUS/ERIC ': 'WOLVERINE/LOGAN ', 'POLARIS/LORNA DANE': 'WOLVERINE/LOGAN ', 'SUMMERS, NATHAN CHRI': 'WOLVERINE/LOGAN ', 'SUNSPOT/ROBERTO DACO': 'WOLVERINE/LOGAN ', 'BANSHEE/SEAN CASSIDY': 'WOLVERINE/LOGAN ', 'BOOMER/TABITHA SMITH': 'WOLVERINE/LOGAN ', 'MACTAGGERT, MOIRA KI': 'WOLVERINE/LOGAN ', 'JUBILE

In [205]:
graph_contracted.nodes
sample = graph_contracted.get_edge_data('SPIDER-MAN/PETER PAR','HULK/DR. ROBERT BRUC')
print(sample)

{'weight': 3417}


In [ ]:
# collapsed graph for visualization

nt = Network(height="1000px", width="100%", bgcolor="#222222", font_color="white", notebook=True, directed = False)

nt.barnes_hut(overlap = 1)
nt.from_nx(graph_contracted)

nt.show("Marvel_Super_Nodes.html")

In [ ]:
hg_sample = nx.from_pandas_edgelist(hero_edges.sample(5000), 'hero1', 'hero2')


net = Network(height="1000px", width="100%", bgcolor="#222222", font_color="white", notebook=True, directed = False)

net.barnes_hut(overlap=1)
net.from_nx(hg_sample)

net.show("Marvel.html")

In [99]:
spec_cluster = algo.r_spectral_clustering(hero_graph,  n_clusters=25, method="vanilla")

In [103]:
spec_coms = pd.DataFrame(spec_cluster.communities).transpose()
spec_coms.head(20)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,"LITTLE, ABNER","ERWIN, CLYTEMNESTRA",PYRO/ALLERDYCE JOHNN,ETERNITY/ADAM QADMON,LIVING LASER/ARTHUR,DEATH,GOG III,"ZIMMER, ABE","JACKSON, STEVE","STARR, TRIXIE TRISH",...,CLOUD,GREEN GOBLIN III/BAR,WONG,"KRIEGHUND, COL.",SHOCKER/HERMAN SCHUL,ASANO,DOMINUS,"KAFKA, DR. ASHLEY",KARMA/XI'AN COY MANH,"TWAKI, DR. TED"
1,PRINCESS ZANDA,IRON MAN IV/JAMES R.,"JONES, LORRAINE LORR",KARNAK [INHUMAN],"CLEMSON, MIKE","BUCKNER, GRANT",KRAKOA,"JONES, DANIEL DANNY",GHOST RIDER III/DAN,"PROUST, PAUL",...,BLOODSTONE/ULYSSES B,SLASHER,"MAJCOMB, RENEE","DELGADO, FATHER","WEIN, LEN",SICKLE,"MARRS(-PAYNE), PHOEB",SCORCHER/,MASTER/ESHU,SISTER MARIA
2,BLACK PANTHER/T'CHAL,"TARR, BLACK JACK",CAPTAIN AMERICA,ULTRON,"GARTH, DONNA",GORGILLA,"KORVAC, MICHAEL","BYRNES, GAYLE WATSON",VOLSTAGG,SILVER DAGGER/ISAIAH,...,"FURY, COL. NICHOLAS",MAX,"TENG, NIKKI","RAHN, TAMARA",IRON FIST H'YLTHRI I,"DAVIS, PRINCIPAL","DILLON, JONATHAN","PEEL, EMMA",RICOCHET/SEN. STEVEN,TANGERINE 2013
3,"STEELE, SIMON/WOLFGA","SMITH, SIR DENIS NAY",ANDROMEDA/ANDROMEDA,DAZZLER II/ALLISON B,MOLE MAN/HARVEY RUPE,"VAUGHAN, MRS.",LEADER/SAM STERNS,"MANNING, DR.","IONELLO, JASON",HOBGOBLIN II/RODERIC,...,STRONG GUY/GUIDO CAR,"MICHEL, RAFE",LUMP,"BECKMAN, SARAH","CARTER, SHARON II","SANADA, KONO","BEERE, JASON","SEVILLE, STACY",SWEEPZWEAK,DE LA COURTE
4,"FORTUNE, DOMINIC","ROM, SPACEKNIGHT","PARKER, MAY",MACHETE/FERDINAND LO,MINERVA [KREE],MONTANA,OX III,"DUGAN, TIMOTHY ALOYI",MAKKARI/MIKE KHARY/I,THIN MAN/BRUCE DICKS,...,TUSK,KID COLT II/ELRIC FR,MAXXAM,"CASSADA, MICHAEL","THOMAS, KIM HO TWAE",AMAZO-MAXI-WOMAN/,"REANDEAU, DR.",FANDRAL | MUTANT X-V,NECRODAMUS,BETA RAY BILL | MUTA
5,IRON MAN/TONY STARK,BLOB/FRED J. DUKES,JOCASTA,ZARAN/MAXIMILLIAN ZA,SUPREME INTELLIGENCE,"EVANS, DORIS",CORRUPTOR/JACKSON DA,JUSUR,"ZINOVIEV, IVAN",PATRIOT/JEFF MACE,...,TRITON DOPPELGANGER,AZIR,VENUS/APHRODITE/VICT,TENPIN/ALVIN HEALEY,TRAPSTER II/LARRY CU,"DEVEREAUX, IRIS",MECHAMAGE/,"VIENNEAU, JACQUES",CURSED LOTUS,None
6,"RAVEN, SABBATH II/EL",TORPEDO III/BROCK JO,GARGOYLE II/ISAAC CH,SILVERMANE/SILVIO MA,PRINCESS PYTHON/ZELD,CERISE II [SHI'AR],MIDGARD SERPENT,WHIRLWIND/DAVID CANN,PERRIKUS,GROTTU,...,GHOST GIRL/,SPOILSPORT/,PSI-WOLF/NIKOLAI VON,"BRADDOCK, JAMES","CAMPBELL, VALERIE","LUMPKIN, WILLIE | TI",WARFIST,None,None,None
7,CARNIVORE/COUNT ANDR,HYBRID/JAMES JIMMY M,COBRA/KLAUS VORHEES,SCARECROW II,DR. NEMESIS/DR. STOC,OJANI,BUZZ,"STARK, HOWARD","ROSS, COURTNEY","GAMBONNO, LUIGI",...,"DEVERAUX, INSPECTOR",MALPRACTICE,BENGAL/,CACTUS,"REYES, DR. DANIEL","SCICLUNA, MICHELE",KILLER CLOWN,None,None,None
8,GHOST,"KILLBURN, MACK",MALICE IV,PENANCE/MONET ST. CR,"LEWIS, DEVON","MCNAIR, TAD",IGOR,PLUTO [OLYMPIAN],CONDOR,BLACKBODY,...,ARIA,"FAWN, FELICIA","FORD, GERALD",SHADOW QUEEN/SHIALMA,"CROWLEY, SEN.","KANE, SUGAR",None,None,None,None
9,FU MANCHU,"CLARK, JOHN","RAVEN, LT. JACOB",UNUS THE UNTOUCHABLE,THUNDERBIRD II/JAMES,KAMIKAZE,TUVO,"REYES, DR. CECELIA","TAKACHIHO, HIRO",QUICKSAND,...,None,None,None,None,None,None,None,None,None,None


In [110]:
# compress graph into supernodes for better visualization

# hero_graph_contracted = contracted_nodes(hero_graph, )

In [122]:

nx.draw_spectral(hero_graph, with_labels = True)

KeyboardInterrupt: 